In [293]:
import numpy as np
import deepdish as dd
import h5py
from collections import Counter

acquire data from HDF5 file

In [300]:
filename = '/Users/lijin/Documents/Studing_in_Uni_HD/internship/overlap_tracking/Labels_1312.h5'

In [301]:
f = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % f.keys())
a_group_key = list(f.keys())[0]

# Get the data
data = list(f[a_group_key])

Keys: KeysView(<HDF5 file "Labels_1312.h5" (mode r)>)


In [272]:
# convert data from list to numpy
data = np.array(data)

In [273]:
data.shape

(2100, 840, 840, 1)

In [274]:
data[2020][data[2020] == 4]

array([4], dtype=uint8)

validate all labeled data in order to figure out non-correct data from them

In [275]:
def get_labels(data):
    counter = []
    labeled_data = []
    for i in range(len(data)):
        if data[i][data[i]!=0].size != 0 and i % 20 == 0:
            labeled_data.append(data[i][data[i]!=0])
            counter.append(i)
    labeled_data = np.array(labeled_data)
    counter = np.array(counter)
    return labeled_data, counter

In [276]:
labels, counter = get_labels(data)

In [277]:
labeled_data = data[counter].reshape(len(counter), 840, -1)

In [278]:
labeled_data.shape

(105, 840, 840)

duplicate labels

In [281]:
labels

array([array([1, 2, 4, 7, 3, 6, 8, 5], dtype=uint8),
       array([1, 7, 8, 2, 4, 3, 6, 5], dtype=uint8),
       array([1, 8, 7, 2, 5, 4, 3, 6], dtype=uint8),
       array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 1, 8, 2, 7, 4, 5, 3, 6], dtype=uint8),
       array([1, 2, 8, 5, 7, 3, 6, 4], dtype=uint8),
       array([1, 5, 2, 6, 3, 4, 8, 7], dtype=uin

indeces for labeled data

In [282]:
counter

array([   0,   20,   40,   60,   80,  100,  120,  140,  160,  180,  200,
        220,  240,  260,  280,  300,  320,  340,  360,  380,  400,  420,
        440,  460,  480,  500,  520,  540,  560,  580,  600,  620,  640,
        660,  680,  700,  720,  740,  760,  780,  800,  820,  840,  860,
        880,  900,  920,  940,  960,  980, 1000, 1020, 1040, 1060, 1080,
       1100, 1120, 1140, 1160, 1180, 1200, 1220, 1240, 1260, 1280, 1300,
       1320, 1340, 1360, 1380, 1400, 1420, 1440, 1460, 1480, 1500, 1520,
       1540, 1560, 1580, 1600, 1620, 1640, 1660, 1680, 1700, 1720, 1740,
       1760, 1780, 1800, 1820, 1840, 1860, 1880, 1900, 1920, 1940, 1960,
       1980, 2000, 2020, 2040, 2060, 2080])

remove duplicate labels from labeled data 

In [283]:
def remove_duplicates(labeled_data, labels, counter):
    for i in range(len(labels)):
        duplicate_label = [item for item, count in Counter(labels[i]).items() if count > 1]
        if duplicate_label:
            duplicates = labeled_data[i][labeled_data[i] == duplicate_label]
            duplicate_coordinates = np.where(labeled_data[i,:] == duplicate_label)
            x = duplicate_coordinates[0][1:]
            y = duplicate_coordinates[1][1:]
            labeled_data[i][x,y] = 0
            #labeled_data[i][duplicate_loc[1:]] = 0
    return labeled_data

In [284]:
nondul_labeled_data = remove_duplicates(labeled_data, labels, counter)

In [285]:
nondul_labeled_data.shape

(105, 840, 840)

check if labels are complete for each piece of non-duplicate labeled data

In [286]:
def check_completeness(data, counter):
    label = np.array([1,2,3,4,5,6,7,8])
    for i in range(len(data)):
        if np.isin(label, data[i]).sum(0) != len(label):
            print(data[i][data[i] != 0])
            print(counter[i])

In [287]:
check_completeness(nondul_labeled_data, counter)

fix the dataset based on this completeness information using ilastik

In [288]:
for i in range(len(nondul_labeled_data)):
    print(nondul_labeled_data[i][nondul_labeled_data[i] != 0])

[1 2 4 7 3 6 8 5]
[1 7 8 2 4 3 6 5]
[1 8 7 2 5 4 3 6]
[3 1 8 2 7 4 5 6]
[1 2 8 5 7 3 6 4]
[1 5 2 6 3 4 8 7]
[5 3 6 1 2 4 8 7]
[3 6 5 4 2 1 8 7]
[4 3 2 6 1 5 7 8]
[4 3 2 1 6 5 8 7]
[2 4 1 3 6 7 8 5]
[1 2 7 4 6 5 8 3]
[1 2 7 5 6 3 8 4]
[2 1 8 7 3 4 6 5]
[2 1 3 7 8 4 5 6]
[2 1 3 4 8 7 5 6]
[2 1 5 4 6 3 8 7]
[5 2 6 1 4 7 8 3]
[5 6 2 1 4 7 3 8]
[5 6 7 2 3 1 8 4]
[5 6 7 8 3 2 4 1]
[5 6 7 8 3 4 2 1]
[5 6 7 8 4 3 1 2]
[5 6 7 8 2 1 4 3]
[7 5 6 8 2 4 3 1]
[5 6 7 8 2 1 3 4]
[5 6 7 8 2 1 3 4]
[7 6 5 8 2 3 1 4]
[8 7 6 5 1 2 3 4]
[8 7 6 5 1 2 3 4]
[8 7 6 5 2 3 1 4]
[8 7 6 3 1 2 5 4]
[8 7 1 3 2 6 4 5]
[1 8 3 2 7 4 5 6]
[3 1 4 2 7 8 6 5]
[3 4 1 7 2 8 5 6]
[3 4 7 2 1 8 5 6]
[3 4 8 7 2 1 5 6]
[4 3 8 7 2 1 5 6]
[3 4 5 6 8 2 7 1]
[3 4 5 6 8 2 7 1]
[4 3 5 6 2 8 1 7]
[5 4 6 3 2 8 7 1]
[5 6 4 3 1 2 7 8]
[6 5 1 4 3 2 7 8]
[6 1 5 2 7 8 4 3]
[1 2 6 7 5 8 4 3]
[1 2 8 7 6 5 4 3]
[1 2 8 7 6 5 4 3]
[1 2 8 7 6 5 3 4]
[2 1 8 7 6 3 5 4]
[2 1 8 7 6 3 5 4]
[2 1 8 7 6 5 3 4]
[2 1 8 6 7 3 5 4]
[2 1 8 3 7 4 5 6]
[3 4 2 1 7

save non-duplicate labeled data and its indeces as '.h' files

In [297]:
save_path_name_first = 'clean_labeled_data.h5'
save_path_name_second = 'clean_labeled_data_indeces.h5'
dd.io.save(save_file_name_first, nondul_labeled_data)
dd.io.save(save_file_name_second, counter)